In [1]:
using JuMP, GLPK
using Printf

The original/main problem we are trying to solve is,

\begin{equation} \nonumber
\begin{align*}
& \text{(Main)} \quad  & \min \quad & c_1^T x + c_2^T y \\
&& \text{s.t. \quad} & A_1 x + A_2 y \leq b \\
&&& x \in \mathbb Z_+^{n}\\
&&& y \geq 0
\end{align*}
\end{equation}

The Sub Problem (SP) is,
\begin{equation} \nonumber
\begin{align*}
& \text{(SP)} \quad  & \min \quad & c_2^T y \\
&& \text{s.t. \quad} & A_2 y \leq b - A_1 x\\
&&& y \geq 0
\end{align*}
\end{equation}
We define $\theta = c_2^T y$ and $\theta_k$ is the optimal objective value for a given $x$ ast iteration $k$.

The Dual of the Sub Problem (DSP) is,
\begin{equation} \nonumber
\begin{align*}
& \text{(DSP)} \quad  & \max \quad & (b - A_1 x)^T \lambda \\
&& \text{s.t. \quad} & A_2^T  \lambda \geq C_2 \\
&&& \lambda \geq 0
\end{align*}
\end{equation}


The Relaxed Main Problem (RMP) is,

\begin{equation} \nonumber
\begin{align*}
& \text{(RMP)} \quad  & \min \quad & c_1^T x + \theta \\
&& \text{s.t. \quad} & \theta \geq \theta^k - \lambda_k^T A_1 (x - x_k)\\
&&& x \in \mathbb Z_+^{n}\\
\end{align*}
\end{equation}
Here $x_k$ is the solution vector of the first stage problem at iteration $k$ and similarly $\lambda_k$ is the solution vector of DSP at iteration $k$. Notice that variable $\theta$ could be negative. At each iteration we add new cuts of the form $\theta \geq \theta^k - \lambda_k^T A_1 (x - x_k)$ to our RMP.

# Data

In [9]:
c1 = [1, 4]
c2 = [2, 3]
b2 = [-2; -3]
A1= [1 -3; -1 -3]
A2= [1 -2; -1 -1]
M = -1000;

# Solve the original problem directly 
Since this is a small problem for illustration purposes we can directly solve the problem without decomposition.

In [10]:
main = Model(GLPK.Optimizer)
@variable(main, 0 <= x[1:2], Int)
@variable(main, 0 <= y[1:2])
@objective(main, Min, c1' * x + c2' * y)
@constraint(main, A1 * x + A2 * y .<= b2);

In [11]:
optimize!(main)

In [12]:
# final solution
objective_value(main), value.(x), value.(y)

(4.0, [0.0, 1.0], [0.0, 0.0])

# Solve Benders decomposition

In [13]:
rmp = Model(GLPK.Optimizer)
@variable(rmp, 0 <= x[1:2], Int)
@variable(rmp, -1000 <= θ) # \theta is a continuous variable
@objective(rmp, Min, c1' * x + θ);


In [14]:
function sub(x)
    sub = Model(GLPK.Optimizer)
    @variable(sub, 0 <= y[1:2])
    @objective(sub, Min, c2' * y)
    @constraint(sub, A1 * x + A2 * y .<= b2)
    optimize!(sub)
    o = objective_value(sub)
    y = value.(y)
    all_con = all_constraints(sub, AffExpr, MOI.LessThan{Float64})
    λ = dual.(all_con)
    return Dict("o"=>o, "y" => y, "λ" => λ)
end

sub (generic function with 1 method)

In [15]:
sub([0, 0])

Dict{String, Any} with 3 entries:
  "λ" => [-0.333333, -2.33333]
  "o" => 7.66667
  "y" => [1.33333, 1.66667]

In [16]:
function print_iteration(k, args...)
    f(x) = Printf.@sprintf("%12.4e", x)
    println(lpad(k, 9), " ", join(f.(args), " "))
    return
end

print_iteration (generic function with 1 method)

In [17]:
println("k  lowerbound  upperbound      gap")
for k in 1:10
    optimize!(rmp)
    lb = objective_value(rmp)
    x_k = value.(x)
    ub = c1' * x_k + c2' * sub(x_k)["y"]
    gap = (ub - lb)/ ub
    print_iteration(k, lb, ub, gap)
    if gap < 0.0000001
        println("**We solved the problem to optimimality.**")
        break
    end
    benderscut = @constraint(rmp, θ >= sub(x_k)["o"] - (sub(x_k)["λ"])' * A1 * (x .- x_k))
    @info "We are adding this bender cut $(benderscut)"
end

k  lowerbound  upperbound      gap


        1  -1.0000e+03   7.6667e+00   1.3143e+02


        2  -4.9600e+02   1.2630e+03   1.3927e+00
        3  -1.0800e+02   8.8800e+02   1.1216e+00
        4   4.0000e+00   4.0000e+00   0.0000e+00
**We solved the problem to optimimality.**


┌ Info: We are adding this bender cut 2 x[1] + 8 x[2] + θ ≥ 7.666666666666666
└ @ Main /home/vahid/code/OptVault/examples/benders_decomposition/benders_decomposition.ipynb:14
┌ Info: We are adding this bender cut -1.5 x[1] + 4.5 x[2] + θ ≥ 3
└ @ Main /home/vahid/code/OptVault/examples/benders_decomposition/benders_decomposition.ipynb:14
┌ Info: We are adding this bender cut θ ≥ 0
└ @ Main /home/vahid/code/OptVault/examples/benders_decomposition/benders_decomposition.ipynb:14


In [18]:
# final solution
objective_value(rmp), value.(x), value.(θ)
# Benders decomposition solution matches the solution we got from solving the orginal problem directly which verifies our Benders implementation.

(4.0, [0.0, 1.0], 0.0)

# Solve Benders decomposition (second version)
In some textbooks/papers you notice that the Benders cut is slightly different. Particularly instead of this cut,

$\theta \leq \theta_k - \lambda_k^T  A (x - x_k) \quad (1)$,

you would see this cut,

$\theta \leq \lambda_k^T (b - A x) \quad (2)$,

where $\theta_k$ is the solution to the subproblem primal problem at iteration $k$ and $\lambda_k$ is the dual solution vector.

These cuts are equivalent and you should get the same solution. Why are these cuts equivalent?
We know that when we solve the subproblem's dual to optimality the strong duality holds between the subproblem and its dual,

$\theta_k = \lambda_k^T  (b - A  x_k) \quad(3)$.

If we subtract (3) from (1) we get (2),

Notice how in constraint (1) we don't see right-hand-side vector $b$.


In [19]:
rmp2 = Model(GLPK.Optimizer)
@variable(rmp2, 0 <= x[1:2], Int)
@variable(rmp2, -1000 <= θ) # \theta is a continuous variable
@objective(rmp2, Min, c1' * x + θ);


In [20]:
println("k  lowerbound  upperbound      gap")
for k in 1:10
    optimize!(rmp2)
    lb = objective_value(rmp2)
    x_k = value.(x)
    ub = c1' * x_k + c2' * sub(x_k)["y"]
    gap = (ub - lb)/ ub
    print_iteration(k, lb, ub, gap)
    if gap < 0.0000001
        println("**We solved the problem to optimimality.**")
        break
    end
    benderscut = @constraint(rmp2, θ >=  (sub(x_k)["λ"])' * (b2 - A1 * x))
    @info "We are adding this bender cut $(benderscut)"
end

k  lowerbound  upperbound      gap


        1  -1.0000e+03   7.6667e+00   1.3143e+02
        2  -4.9600e+02   1.2630e+03   1.3927e+00
        3  -1.0800e+02   8.8800e+02   1.1216e+00
        4   4.0000e+00   4.0000e+00   0.0000e+00
**We solved the problem to optimimality.**


┌ Info: We are adding this bender cut 2 x[1] + 8 x[2] + θ ≥ 7.666666666666666
└ @ Main /home/vahid/code/OptVault/examples/benders_decomposition/benders_decomposition.ipynb:14
┌ Info: We are adding this bender cut -1.5 x[1] + 4.5 x[2] + θ ≥ 3
└ @ Main /home/vahid/code/OptVault/examples/benders_decomposition/benders_decomposition.ipynb:14
┌ Info: We are adding this bender cut θ ≥ 0
└ @ Main /home/vahid/code/OptVault/examples/benders_decomposition/benders_decomposition.ipynb:14


In [21]:
# final solution
objective_value(rmp2), value.(x), value.(θ)
# Benders decomposition solution matches the solution we got from solving the orginal problem directly which verifies our Benders implementation.

(4.0, [0.0, 1.0], 0.0)